In [20]:
import cv2 
import numpy as np 
import imutils
import math

In [23]:

# Let's load a simple image with 3 black squares 
image = cv2.imread('disparity.png') 
cv2.waitKey(0) 

# Grayscale 
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 

# Find Canny edges 
edged = cv2.Canny(gray, 30, 200) 
cv2.waitKey(0) 

# Finding Contours 
# Use a copy of the image e.g. edged.copy() 
# since findContours alters the image 
contours, hierarchy = cv2.findContours(edged, 
	cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 

cv2.imshow('Canny Edges After Contouring', edged) 
cv2.waitKey(0) 

print("Number of Contours found = " + str(len(contours))) 

# Draw all contours 
# -1 signifies drawing all contours 
cv2.drawContours(image, contours, -1, (0, 0, 255), 3) 

cv2.imshow('Contours', image) 
cv2.waitKey(0) 
cv2.destroyAllWindows() 


Number of Contours found = 7


In [28]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
thresh = cv2.threshold(blurred, 60, 255, cv2.THRESH_BINARY_INV)[1]
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
#print(cnts)
m=0
for c in cnts:
    #print(c)
    #print(len(c))
    #continue
    area=cv2.contourArea(c)
    print(area)
    if area>m:
        #print(c)
        
        m=area
        M = cv2.moments(c)
        cX = int(M["m10"] / (M["m00"] ))
        cY = int(M["m01"] / (M["m00"]))
    #print(len(c))
    # compute the center of the contour
        print(cX,cY)

color = np.array(image[cY][cX])
cv2.circle(image, (int(cX), int(cY)), 5, (255, 0, 0), -1)
cv2.imshow('Contours', image) 
cv2.waitKey(0) 
cv2.destroyAllWindows() 

0.5
53 133
0.5
56 132
1.0
55 122
0.0
0.0
0.0
1.5
7.0
9.5
195 80
17.0
26 77
0.0
3.0
4.0
2848.5
225 20
3248.5


In [19]:
image.shape

(480, 640, 3)

In [10]:
print(image[125][192]) # Let this distance be 2 m
print(image[110][116]) # Let this distance be 4 m

[68 68 68]
[95 95 95]


In [11]:
color

array([66, 66, 66], dtype=uint8)

In [12]:
# Let 160 -> 2 m, 121 -> 4m
# By Linear Regression, y = x*-0.02820855 + 7.03992702

def colorToDistance(color, slope = -0.02820855, constant = 7.03992702):
    # Simple Linear Regression
    return color*slope + constant

In [13]:
def pixelToComponents(pixel, distance, heading = 0, currentLocation = (0,0,0), image_size = image.shape[:2], max_angleX = math.radians(90)):
    # heading = 0 means heading towards X axis, 90 means Y axis and so on
    # max_x / 2 is equal to heading angle. if this - x = 0, then X,Y = distance*cos(heading), distance*sin(heading)
    # AngleXY 
    max_y, max_x = image_size
    x,y = pixel
    mid_x = max_x / 2
    mid_y = max_y / 2
    deltaAngle = ((x - mid_x) / max_x) * max_angleX
    angleXY = heading - deltaAngle
    angleZ = -((y - mid_y) / max_y) * max_angleX
    return np.multiply(distance, [math.cos(angleXY), math.sin(angleXY), math.sin(angleZ)])

In [14]:
dis = colorToDistance(color)
print(dis)
print(pixelToComponents((cX,cY), dis))

[5.17816272 5.17816272 5.17816272]
[5.03804139 1.19645648 3.41419992]


In [15]:
pixelToComponents((200, 194), dis)

array([4.95519277, 1.50314129, 0.77655226])